In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
FILENAME = '../data/statistics.csv'
if FILENAME[-3:]=="csv":
    df = pd.read_csv(FILENAME,index_col=0)

In [16]:
df.tail(10)

,UUID,HOSTNAME,ALIAS,TIMESTAMP,STREAMS,PACING,THROUGHPUT (Sender),THROUGHPUT (Receiver),LATENCY (min.),LATENCY (max.),LATENCY (mean),RETRANSMITS,CONGESTION (Sender),CONGESTION (Receiver),BYTES (Receiver)
64,24c50992-c934-4751-9cca-ed6675a13a73,10.201.1.2,hostB,2021-06-28T21:46:43.042Z,4,3gbit,2.468477e+09,2.422197e+09,50147.0,51812.5,51065.0,193,bbr2,bbr2,18197435688
65,2baa70c4-b5d6-4682-af95-bf0a4db885c7,10.201.1.2,hostB,2021-06-28T21:47:56.826Z,4,4gbit,2.608295e+09,2.566061e+09,50162.0,52041.0,51321.0,263,bbr2,bbr2,19278225668
66,b1147fc9-4825-4d75-bf5a-602a5a24e0be,10.201.1.2,hostB,2021-06-28T21:49:10.636Z,4,5gbit,3.014615e+09,2.970174e+09,50141.0,52200.0,51343.0,252,bbr2,bbr2,22314598396
67,dc30d56a-fac1-465e-ab87-55053e8f31e2,10.201.1.2,hostB,2021-06-28T21:50:24.435Z,4,6gbit,2.308398e+09,2.263195e+09,50161.0,52218.5,51233.0,212,bbr2,bbr2,17002958220
68,e8e54987-2a2e-48a7-8b80-dc79da97152f,10.201.1.2,hostC,2021-06-28T21:51:38.841Z,4,1gbit,9.725254e+08,9.651330e+08,20177.5,21015.0,20775.0,0,bbr2,bbr2,7243374620
69,a3acc420-bdd7-4c20-89f8-7354dab4cfbb,10.201.1.2,hostC,2021-06-28T21:52:51.836Z,4,2gbit,1.941758e+09,1.927249e+09,20183.5,20886.5,20719.0,0,bbr2,bbr2,14464124284
70,43b9764e-9da4-4220-9295-1346cc21c86b,10.201.1.2,hostC,2021-06-28T21:54:04.818Z,4,3gbit,2.569156e+09,2.551623e+09,20240.5,21201.5,20981.5,0,bbr2,bbr2,19150591396
71,a863c7d7-331c-441d-9179-8b9cf2cdfc8a,10.201.1.2,hostC,2021-06-28T21:55:17.810Z,4,4gbit,2.891742e+09,2.870526e+09,20475.5,21328.0,21136.0,0,bbr2,bbr2,21544130856
72,247c4b03-41d0-4360-9e25-eada6a2bb4e1,10.201.1.2,hostC,2021-06-28T21:56:30.798Z,4,5gbit,2.680630e+09,2.658751e+09,20250.5,21300.5,21089.5,0,bbr2,bbr2,19954831836
73,2e01ab26-4733-4a53-85d3-4d45d059db74,10.201.1.2,hostC,2021-06-28T21:57:43.798Z,4,6gbit,2.051860e+09,2.030815e+09,20207.5,21386.0,21128.5,0,bbr2,bbr2,15241839704


In [15]:
df['HOSTNAME']

0     10.201.1.2
1     10.201.1.2
2     10.201.1.2
3     10.201.1.2
4     10.201.1.2
         ...    
69    10.201.1.2
70    10.201.1.2
71    10.201.1.2
72    10.201.1.2
73    10.201.1.2
Name: HOSTNAME, Length: 74, dtype: object